In [ ]:
# default_exp experiment.mining.ir.unsupervised.d2v

# Neural Unsupervised Approaches for SE Traceability [approach d2v]

> Just Paper. Full Experimentation. This module is dedicated to evaluate word2vec/doc2vec or any neural unsupervised approaches on traceability datasets. Consider to Copy the entire notebook for a new and separeted empirical evaluation. 
> Implementing mutual information analysis
> Author: @danaderp April 2020
> Author: @danielrc Nov 2020

In [ ]:
from ds4se.mining.ir import Doc2VecSeqVect as d2v
from ds4se.mining.ir import VectorizationType
from ds4se.mining.ir import LinkType
from ds4se.mining.ir import SoftwareArtifacts
from ds4se.mining.ir import Preprocessing
from ds4se.mining.ir import DistanceMetric
from ds4se.mining.ir import LoadLinks
from ds4se.mining.ir import SimilarityMetric

In [ ]:
from ds4se.mgmnt.prep.conv import *

In [ ]:
from prg import prg

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# Artifacts Similarity with BasicSequenceVectorization

We test diferent similarities based on [blog](https://www.kdnuggets.com/2017/08/comparing-distance-measurements-python-scipy.html) and [blog2](https://www.kdnuggets.com/2019/01/comparison-text-distance-metrics.html)

# Experients Set-up

In [ ]:
path_data = '../dvc-ds4se/' #dataset path

In [ ]:
path_model_prefix = path_data+'models/bpe/sentencepiece/wiki_py_java_bpe_8k'
path_to_trained_model = path_data+'/models/pv/bpe8k/[doc2vec-Py-Java-PVDBOW-500-20E-8k-1594572857.17191].model'
def get_params():
        return {
        "vectorizationType": VectorizationType.doc2vec,
        "linkType": LinkType.req2tc,
        "system": 'libest',
        "path_to_trained_model": path_to_trained_model,
        "source_type": SoftwareArtifacts.REQ.value,
        "target_type": SoftwareArtifacts.TC.value,
        "system_path_config": {
            "system_path": path_data + 'se-benchmarking/traceability/cisco/libest_data/[libest-all-corpus-1596063103.098236].csv',
            "sep": '~',
            "names": ['ids','bpe8k'],
            "prep": Preprocessing.bpe
        },
        "path_mappings": path_data + "se-benchmarking/traceability/testbeds/groundtruth/english/[libest-ground-req-to-tc].txt",
        "saving_path": path_data + 'metrics/traceability/experiments1.0.x/',
        "names": ['Source','Target','Linked?'],
        "model_prefix": path_model_prefix
        }

In [ ]:

parameters = get_params()
parameters

{'vectorizationType': <VectorizationType.doc2vec: 2>,
 'linkType': <LinkType.req2tc: 1>,
 'system': 'libest',
 'path_to_trained_model': '../dvc-ds4se//models/pv/bpe8k/[doc2vec-Py-Java-PVDBOW-500-20E-8k-1594572857.17191].model',
 'source_type': 'req',
 'target_type': 'tc',
 'system_path_config': {'system_path': '../dvc-ds4se/se-benchmarking/traceability/cisco/libest_data/[libest-all-corpus-1596063103.098236].csv',
  'sep': '~',
  'names': ['ids', 'bpe8k'],
  'prep': <Preprocessing.bpe: 2>},
 'path_mappings': '../dvc-ds4se/se-benchmarking/traceability/testbeds/groundtruth/english/[libest-ground-req-to-tc].txt',
 'saving_path': '../dvc-ds4se/metrics/traceability/experiments1.0.x/',
 'names': ['Source', 'Target', 'Linked?'],
 'model_prefix': '../dvc-ds4se/models/bpe/sentencepiece/wiki_py_java_bpe_8k'}

# Artifacts Similarity with Doc2Vec

In [ ]:
#[step 1]Creating the Vectorization Class
doc2vec = d2v( params = parameters, logging = logging )

2020-12-23 02:30:05,765 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-12-23 02:30:05,899 : INFO : built Dictionary(3113 unique tokens: ['\n', '\n\n', '!', '"', '")']...) from 87 documents (total 468327 corpus positions)
2020-12-23 02:30:05,950 : INFO : bpe preprocessing documents, dictionary, and vocab for the test corpus
2020-12-23 02:30:05,951 : INFO : loading Doc2Vec object from ../dvc-ds4se//models/pv/bpe8k/[doc2vec-Py-Java-PVDBOW-500-20E-8k-1594572857.17191].model
2020-12-23 02:30:06,016 : INFO : loading vocabulary recursively from ../dvc-ds4se//models/pv/bpe8k/[doc2vec-Py-Java-PVDBOW-500-20E-8k-1594572857.17191].model.vocabulary.* with mmap=None
2020-12-23 02:30:06,017 : INFO : loading trainables recursively from ../dvc-ds4se//models/pv/bpe8k/[doc2vec-Py-Java-PVDBOW-500-20E-8k-1594572857.17191].model.trainables.* with mmap=None
2020-12-23 02:30:06,018 : INFO : loading syn1neg from ../dvc-ds4se//models/pv/bpe8k/[doc2vec-Py-Java-PVDBOW-500-20E-8k-1594572857.17

In [ ]:
doc2vec.df_source.head(2)

,ids,bpe8k
35,test_data/LibEST_semeru_format/requirements/RQ...,"['▁require', 'ment', '▁17', ':', '▁http', '▁ur..."
36,test_data/LibEST_semeru_format/requirements/RQ...,"['▁require', 'ment', '▁4', '6', ':', '▁ser', '..."


In [ ]:
str(type(doc2vec.new_model))

"<class 'gensim.models.doc2vec.Doc2Vec'>"

In [ ]:
#[step1]Apply Doc2Vec Inference
doc2vec.InferDoc2Vec( steps = 200 )

2020-12-23 02:30:15,348 : INFO : Infer Doc2Vec on Source and Target Complete


In [ ]:
doc2vec.df_inferred_src.head(2)

,ids,bpe8k,inf-doc2vec
35,test_data/LibEST_semeru_format/requirements/RQ...,"['▁require', 'ment', '▁17', ':', '▁http', '▁ur...","[9.933052e-05, -2.0830888e-05, -0.00048534758,..."
36,test_data/LibEST_semeru_format/requirements/RQ...,"['▁require', 'ment', '▁4', '6', ':', '▁ser', '...","[-0.00096515525, 0.00022701142, 0.00028692474,..."


In [ ]:
#[step 2]NonGroundTruth Computation
metric_l = [DistanceMetric.EUC,DistanceMetric.COS,DistanceMetric.MAN, SimilarityMetric.Pearson]
doc2vec.ComputeDistanceArtifacts( sampling=False, samples = 50, metric_list = metric_l )
doc2vec.df_nonground_link.head()

2020-12-23 02:30:21,265 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ17.txt', 'test_data/LibEST_semeru_format/test/us903.c')[[0.0184848103672266, 0.9818506764371265], [1.016532814130187, -0.016532814130187035], [0.34296417, 0.7446215026274677], [0.01666422355467995]]
2020-12-23 02:30:21,269 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ17.txt', 'test_data/LibEST_semeru_format/test/us3496.c')[[0.017925607040524483, 0.9823900617918035], [0.9397891722619534, 0.060210827738046646], [0.32319522, 0.7557463824012146], [0.060637146149717706]]
2020-12-23 02:30:21,272 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ17.txt', 'test_data/LibEST_semeru_format/test/us899.c')[[0.018076613545417786, 0.9822443485049062], [0.9950622613541782, 0.00493773864582181], [0.32680115, 0.7536924423213366], [0.004844937790197483]]
2020-12-23 02:30:21,275 : INFO : Computed distanc

2020-12-23 02:30:21,340 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ46.txt', 'test_data/LibEST_semeru_format/test/us1060.c')[[0.018117493018507957, 0.9822049094110019], [1.0476360023021698, -0.0476360023021698], [0.33347625, 0.7499196205760419], [0.04678015435677304]]
2020-12-23 02:30:21,343 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ46.txt', 'test_data/LibEST_semeru_format/test/us900.c')[[0.017628338187932968, 0.9826770368646344], [0.9830556139349937, 0.016944386065006256], [0.3204152, 0.7573375411814196], [0.01661190105674933]]
2020-12-23 02:30:21,345 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ46.txt', 'test_data/LibEST_semeru_format/test/us896.c')[[0.018556348979473114, 0.9817817158588571], [1.0231635700911283, -0.02316357009112835], [0.33931816, 0.7466485803534753], [0.02315176219616641]]
2020-12-23 02:30:21,348 : INFO : Computed distance

2020-12-23 02:30:21,413 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ18.txt', 'test_data/LibEST_semeru_format/test/us898.c')[[0.01828480325639248, 0.9820435273138525], [1.0421390905976295, -0.04213909059762955], [0.33430114, 0.749456001430546], [0.03906268167566914]]
2020-12-23 02:30:21,416 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ18.txt', 'test_data/LibEST_semeru_format/test/us3512.c')[[0.01853269152343273, 0.9818045197000863], [1.0578153729438782, -0.057815372943878174], [0.3376708, 0.7475680845428598], [0.06489611460586177]]
2020-12-23 02:30:21,419 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ18.txt', 'test_data/LibEST_semeru_format/test/us1883.c')[[0.018201792612671852, 0.9821235900931125], [1.0519917532801628, -0.05199175328016281], [0.3310033, 0.7513129334592595], [0.04842864509865108]]
2020-12-23 02:30:21,422 : INFO : Computed distances

2020-12-23 02:30:21,485 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ48.txt', 'test_data/LibEST_semeru_format/test/us901.c')[[0.018373727798461914, 0.9819577751301749], [0.9934775857254863, 0.0065224142745137215], [0.33309382, 0.7501347493291373], [0.005856609291582123]]
2020-12-23 02:30:21,488 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ48.txt', 'test_data/LibEST_semeru_format/test/us1864.c')[[0.01752461865544319, 0.9827772042718729], [0.9619517959654331, 0.03804820403456688], [0.31645805, 0.7596140284964106], [0.05022246771174625]]
2020-12-23 02:30:21,491 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ48.txt', 'test_data/LibEST_semeru_format/test/us1159.c')[[0.017511261627078056, 0.9827901053408724], [0.9436282999813557, 0.05637170001864433], [0.31466532, 0.760649867811754], [0.056882596722068954]]
2020-12-23 02:30:21,494 : INFO : Computed distan

2020-12-23 02:30:21,558 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ42.txt', 'test_data/LibEST_semeru_format/test/us893.c')[[0.0179708581417799, 0.9823463923372185], [0.9737536013126373, 0.02624639868736267], [0.33090663, 0.7513675097770119], [0.02651859621890995]]
2020-12-23 02:30:21,561 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ42.txt', 'test_data/LibEST_semeru_format/test/us895.c')[[0.01837647147476673, 0.9819551295719208], [1.027784263715148, -0.027784263715147972], [0.3388036, 0.7469355534742563], [0.027871823837416418]]
2020-12-23 02:30:21,564 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ29.txt', 'test_data/LibEST_semeru_format/test/us903.c')[[0.01874661073088646, 0.9815983576942289], [1.0603534206748009, -0.06035342067480087], [0.34533554, 0.7433089871797797], [0.060240803926764154]]
2020-12-23 02:30:21,567 : INFO : Computed distances o

...

2020-12-23 02:30:24,276 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ1.txt', 'test_data/LibEST_semeru_format/test/us1060.c')[[0.01774381846189499, 0.9825655355109786], [0.9841941054910421, 0.015805894508957863], [0.32563415, 0.7543559425651023], [0.02090179148038026]]
2020-12-23 02:30:24,278 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ1.txt', 'test_data/LibEST_semeru_format/test/us900.c')[[0.017853869125247, 0.9824593002327625], [0.9878262979909778, 0.012173702009022236], [0.32212418, 0.7563586028631075], [0.010433832056126587]]
2020-12-23 02:30:24,281 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ1.txt', 'test_data/LibEST_semeru_format/test/us896.c')[[0.018511246889829636, 0.9818251914778984], [0.9986665560863912, 0.0013334439136087894], [0.34400183, 0.744046606156504], [0.0014145952850829974]]
2020-12-23 02:30:24,284 : INFO : Computed distances 

2020-12-23 02:30:24,351 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ38.txt', 'test_data/LibEST_semeru_format/test/us898.c')[[0.017601726576685905, 0.982702735149733], [0.9690325390547514, 0.030967460945248604], [0.31265682, 0.7618137390523889], [0.029166660750107227]]
2020-12-23 02:30:24,354 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ38.txt', 'test_data/LibEST_semeru_format/test/us3512.c')[[0.01798732578754425, 0.9823305012430987], [0.9998328729707282, 0.00016712702927179635], [0.32728738, 0.7534163423362665], [0.003751611265440136]]
2020-12-23 02:30:24,357 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ38.txt', 'test_data/LibEST_semeru_format/test/us1883.c')[[0.018017033115029335, 0.9823018353043671], [1.0343388132750988, -0.0343388132750988], [0.32953435, 0.752143033105389], [0.03664896459883762]]
2020-12-23 02:30:24,359 : INFO : Computed dista

2020-12-23 02:30:24,423 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ52.txt', 'test_data/LibEST_semeru_format/test/us901.c')[[0.018302686512470245, 0.9820262808348723], [0.9846067102625966, 0.015393289737403393], [0.33561993, 0.7487159933709342], [0.015206555643039292]]
2020-12-23 02:30:24,426 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ52.txt', 'test_data/LibEST_semeru_format/test/us1864.c')[[0.018417909741401672, 0.9819151749343465], [1.0611991919577122, -0.06119919195771217], [0.34269333, 0.7447717051301356], [0.05844890975480405]]
2020-12-23 02:30:24,428 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ52.txt', 'test_data/LibEST_semeru_format/test/us1159.c')[[0.018141301348805428, 0.9821819414213211], [1.011470315977931, -0.011470315977931023], [0.33147123, 0.7510488954201492], [0.01140191510062426]]
2020-12-23 02:30:24,431 : INFO : Computed dist

2020-12-23 02:30:24,497 : INFO : Computed distances or similarities ('test_data/LibEST_semeru_format/requirements/RQ6.txt', 'test_data/LibEST_semeru_format/test/us895.c')[[0.01858958788216114, 0.9817496780810293], [1.0401074439287186, -0.04010744392871857], [0.3410153, 0.7457036420222349], [0.03972865704621621]]
2020-12-23 02:30:24,501 : INFO : Non-groundtruth links computed


,Source,Target,DistanceMetric.EUC,SimilarityMetric.EUC_sim,DistanceMetric.COS,SimilarityMetric.COS_sim,DistanceMetric.MAN,SimilarityMetric.MAN_sim,SimilarityMetric.Pearson
0,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us903.c,0.018485,0.981851,1.016533,-0.016533,0.342964,0.744622,0.016664
1,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us3496.c,0.017926,0.982390,0.939789,0.060211,0.323195,0.755746,0.060637
2,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us899.c,0.018077,0.982244,0.995062,0.004938,0.326801,0.753692,0.004845
3,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us4020.c,0.018608,0.981732,1.039151,-0.039151,0.345708,0.743103,0.039316
4,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us897.c,0.017502,0.982799,0.940846,0.059154,0.321392,0.756778,0.059351


In [ ]:
#[step 3]Saving Non-GroundTruth Links
doc2vec.SaveLinks()

2020-12-23 02:31:34,282 : INFO : Saving in...../dvc-ds4se/metrics/traceability/experiments1.0.x/[libest-VectorizationType.doc2vec-LinkType.req2tc-False-1608690694.259713].csv


In [ ]:
#[step 4]GroundTruthMatching Testing
path_to_ground_truth = parameters['path_mappings']

doc2vec.MatchWithGroundTruth(path_to_ground_truth, semeru_format= True)
doc2vec.df_ground_link

2020-12-23 02:32:21,801 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,807 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,811 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,817 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,821 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,824 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,828 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,832 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,836 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,838 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,841 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,845 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,848 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,852 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,855 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,858 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:21,861 

2020-12-23 02:32:22,225 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,228 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,230 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,233 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,236 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,239 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,242 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,245 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,248 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,251 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,254 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,257 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,259 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,262 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,265 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,268 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,271 

2020-12-23 02:32:22,656 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,659 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,662 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,665 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,668 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,671 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,673 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,676 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,679 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,682 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,685 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,688 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,690 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,693 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,696 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,699 : INFO : findDistInDF: semeru_format
2020-12-23 02:32:22,705 

,Source,Target,DistanceMetric.EUC,SimilarityMetric.EUC_sim,DistanceMetric.COS,SimilarityMetric.COS_sim,DistanceMetric.MAN,SimilarityMetric.MAN_sim,SimilarityMetric.Pearson,Linked?
0,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us903.c,0.018485,0.981851,1.016533,-0.016533,0.342964,0.744622,0.016664,0.0
1,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us3496.c,0.017926,0.982390,0.939789,0.060211,0.323195,0.755746,0.060637,0.0
2,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us899.c,0.018077,0.982244,0.995062,0.004938,0.326801,0.753692,0.004845,0.0
3,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us4020.c,0.018608,0.981732,1.039151,-0.039151,0.345708,0.743103,0.039316,0.0
4,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us897.c,0.017502,0.982799,0.940846,0.059154,0.321392,0.756778,0.059351,0.0
...,...,...,...,...,...,...,...,...,...,...
1087,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us1864.c,0.017553,0.982750,0.962431,0.037569,0.313136,0.761536,0.025255,0.0
1088,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us1159.c,0.018409,0.981924,1.040071,-0.040071,0.335135,0.748988,0.040568,1.0
1089,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us2174.c,0.018371,0.981960,1.037674,-0.037674,0.332956,0.750212,0.036117,0.0
1090,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us893.c,0.019193,0.981168,1.098556,-0.098556,0.353627,0.738756,0.100956,1.0


In [ ]:
#[step 4.1]GroundTruthMatching Testing For CISCO Mappings <--- Only SACP
doc2vec.MatchWithGroundTruth(from_mappings=True)
doc2vec.df_ground_link

KeyError: 'id_pr'

In [ ]:
#[step 5]Saving GroundTruth Links
doc2vec.SaveLinks(grtruth = True)

2020-12-23 02:32:32,297 : INFO : Saving in...../dvc-ds4se/metrics/traceability/experiments1.0.x/[libest-VectorizationType.doc2vec-LinkType.req2tc-True-1608690752.27334].csv


In [ ]:
#Loading Non-GroundTruth Links (change the timestamp with the assigned in the previous step)
df_glinks = LoadLinks(timestamp=1608690752.27334, params=parameters,grtruth = True, logging = logging)
df_glinks.head()

2020-12-23 02:33:29,809 : INFO : Loading computed links from... ../dvc-ds4se/metrics/traceability/experiments1.0.x/[libest-VectorizationType.doc2vec-LinkType.req2tc-True-1608690752.27334].csv


,Source,Target,DistanceMetric.EUC,SimilarityMetric.EUC_sim,DistanceMetric.COS,SimilarityMetric.COS_sim,DistanceMetric.MAN,SimilarityMetric.MAN_sim,SimilarityMetric.Pearson,Linked?
0,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us903.c,0.018485,0.981851,1.016533,-0.016533,0.342964,0.744622,0.016664,0.0
1,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us3496.c,0.017926,0.982390,0.939789,0.060211,0.323195,0.755746,0.060637,0.0
2,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us899.c,0.018077,0.982244,0.995062,0.004938,0.326801,0.753692,0.004845,0.0
3,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us4020.c,0.018608,0.981732,1.039151,-0.039151,0.345708,0.743103,0.039316,0.0
4,test_data/LibEST_semeru_format/requirements/RQ...,test_data/LibEST_semeru_format/test/us897.c,0.017502,0.982799,0.940846,0.059154,0.321392,0.756778,0.059351,0.0
